 ### Array Shaping

 A pointer variable can be shaped to a multi-dimensional array to facilitate data access. This is achieved by a  _shape-operator_  casted in front of  a pointer (lvalue expression):   `([s_1][s_2]...[s_n])`  _pointer_   where each $s_i$ is an integral-type expression of positive value. The shape-operator can appear in either the  _motion-clause_  of the  `target` ~ `update`  directive or the  `depend`  clause.

 The following example shows the use of the shape-operator in the   `target` ~ `update`  directive. The shape-operator  `([nx][ny+2])`  casts pointer variable $a$ to a 2-dimentional array of size  _nx_ $times$ _(ny+2)_ .  The resulting array is then accessed as array sections (such as  `[0:nx][1]`  and  `[0:nx][ny]` )  in the  `from`  or  `to`  clause for transferring two columns of  noncontiguous boundary data from or to the device.   Note the use of additional parentheses around the shape-operator and $a$ to ensure the correct precedence  over array-section operations.

In [ ]:

/*
* name: array_shaping.1.c
* type: C
* version: omp_5.0
*/
#pragma omp declare target
int do_work(double *a, int nx, int ny);
int other_work(double *a, int nx, int ny);
#pragma omp end declare target

void exch_data(double *a, int nx, int ny);

void array_shaping(double *a, int nx, int ny)
{
   // map data to device and do work
   #pragma omp target data map(a[0:nx*(ny+2)])
   {
      // do work on the device
      #pragma omp target  // map(a[0:nx*(ny+2)]) is optional here
      do_work(a, nx, ny);

      // update boundary points (two columns of 2D array) on the host
      // pointer is shaped to 2D array using the shape-operator
      #pragma omp target update from( (([nx][ny+2])a)[0:nx][1], \
                                      (([nx][ny+2])a)[0:nx][ny] )

      // exchange ghost points with neighbors
      exch_data(a, nx, ny);

      // update ghost points (two columns of 2D array) on the device
      // pointer is shaped to 2D array using the shape-operator
      #pragma omp target update to( (([nx][ny+2])a)[0:nx][0], \
                                    (([nx][ny+2])a)[0:nx][ny+1] )

      // perform other work on the device
      #pragma omp target  // map(a[0:nx*(ny+2)]) is optional here
      other_work(a, nx, ny);
   }
}



 The shape operator is not defined for Fortran.  Explicit array shaping of procedure arguments can be used instead to achieve a similar goal. Below is the Fortran-equivalent of the above example that illustrates the support of transferring two rows of noncontiguous boundary data in the  `target` ~ `update`  directive.  ffreeexample[5.0]{array_shaping}{1}